In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline, AutoTokenizer, AutoModel
import warnings
import os
import json
from datetime import datetime
warnings.filterwarnings('ignore')

2025-05-31 15:04:12.589935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748703852.614515     117 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748703852.621940     117 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
pipe = pipeline("token-classification", model="cahya/NusaBert-ner-v1.3")
tokenizer = AutoTokenizer.from_pretrained("cahya/NusaBert-ner-v1.3")
model = AutoModel.from_pretrained("cahya/NusaBert-ner-v1.3")

Device set to use cuda:0


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/cleaned/cleaned_all_data.csv')
df.head()

/kaggle/input/cleaned/cleaned_all_data.csv


,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764
2,https://www.detik.com/jatim/hukum-dan-kriminal...,"Kriminalitas Surabaya Marak, Pengamat Sebut Pe...",2024-09-05 02:01:00,Sejumlah kejadian kriminalitas kian marak terj...,72,1224
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398


In [6]:
def debug_nusabert_output(text, pipe):
    """
    Debug function untuk melihat output mentah dari NusaBert
    """
    try:
        raw_entities = pipe(text)
        print("Raw NusaBert Output:")
        for entity in raw_entities:
            print(f"  {entity}")
        
        print("\nUnique entity labels found:")
        unique_labels = set()
        for entity in raw_entities:
            original_label = entity['entity']
            cleaned_label = original_label.replace('B-', '').replace('I-', '')
            unique_labels.add(f"{original_label} -> {cleaned_label}")
        
        for label in sorted(unique_labels):
            print(f"  {label}")
            
        return raw_entities
    except Exception as e:
        print(f"Error: {e}")
        return []

# Test dengan satu artikel
test_text = df.iloc[0]['isi_berita']
print("Testing dengan artikel pertama:")
print(f"Text: {test_text[:200]}...")
print("\n" + "="*50)

debug_output = debug_nusabert_output(test_text, pipe)

Testing dengan artikel pertama:
Text: Pelaku pembunuhan dan pemerkosaan AA (14) seorang remaja putri di Kuburan Cina Palembang sudah ditangkap dan ditetapkan tersangka. Keempat tersangka merupakan anak di bawah umur, sama seperti korbanny...

Raw NusaBert Output:
  {'entity': 'B-PER', 'score': 0.9859017, 'index': 5, 'word': 'ĠAA', 'start': 33, 'end': 36}
  {'entity': 'B-CRD', 'score': 0.99209154, 'index': 7, 'word': '14', 'start': 38, 'end': 40}
  {'entity': 'B-LOC', 'score': 0.9789508, 'index': 13, 'word': 'ĠKub', 'start': 65, 'end': 69}
  {'entity': 'I-LOC', 'score': 0.9747813, 'index': 14, 'word': 'uran', 'start': 69, 'end': 73}
  {'entity': 'I-LOC', 'score': 0.99543375, 'index': 15, 'word': 'ĠCina', 'start': 73, 'end': 78}
  {'entity': 'B-GPE', 'score': 0.59830356, 'index': 16, 'word': 'ĠPalembang', 'start': 78, 'end': 88}
  {'entity': 'B-CRD', 'score': 0.5197823, 'index': 23, 'word': 'ĠKeempat', 'start': 130, 'end': 138}
  {'entity': 'B-ORG', 'score': 0.9998621, 'index': 38, 'word

In [ ]:
import re
from collections import defaultdict

CRIME_TYPES = [
    'pembunuhan', 'pemerkosaan', 'penganiayaan', 'pembacokan', 'penikaman',
    'penyiraman air keras', 'penembakan', 'pengeroyokan', 'mutilasi',
    'pencurian', 'curanmor', 'pencurian motor', 'pencurian dengan kekerasan',
    'perampokan', 'begal', 'penjambretan', 'penggelapan',
    'pencurian spesialis bongkar warung', 'bajing loncat', 'pembobolan',
    'kasus sabu', 'peredaran obat-obatan terlarang', 'kasus narkoba',
    'penyalahgunaan narkoba', 'peredaran pil koplo', 'pencabulan', 'sodomi',
    'pelecehan seksual', 'korupsi dana desa', 'korupsi', 'uang palsu',
    'tawuran', 'aksi perusakan', 'pemalsuan dokumen', 'penjualan miras ilegal',
    'perjudian', 'sabung ayam', 'penculikan', 'perdagangan orang',
    'pembakaran', 'penipuan', 'penyelundupan', 'pemalakan', 'gangster',
    'perampokan agen ATM mini'
]

def detect_crime_types(text):
    """
    Deteksi crime types dalam teks menggunakan pattern matching
    """
    text_lower = text.lower()
    detected_crimes = []
    
    for crime in CRIME_TYPES:
        pattern = r'\b' + re.escape(crime.lower()) + r'\b'
        matches = re.finditer(pattern, text_lower)
        
        for match in matches:
            detected_crimes.append({
                'entity': 'CRIMETYPE',
                'word': crime,
                'start': match.start(),
                'end': match.end(),
                'score': 1.0 
            })
    
    return detected_crimes

def extract_entities_nusabert_fixed(text, pipe):
    """
    Extract entities dengan debugging yang lebih baik
    """
    try:
        entities = pipe(text)
        
        all_labels = set([entity['entity'] for entity in entities])
        print(f"All labels found: {all_labels}")
        
        target_entities = ['DATE', 'TIME', 'LOCATION', 'PERSON', 'ORGANIZATION']
        possible_labels = ['DATE', 'TIME', 'LOCATION', 'PERSON', 'ORGANIZATION',
                          'PER', 'LOC', 'ORG', 'MISC', 'GPE']
        
        filtered_entities = []
        
        for entity in entities:
            # Ambil label tanpa prefix B- atau I-
            original_label = entity['entity']
            label = original_label.replace('B-', '').replace('I-', '')
            
            if label in target_entities or label in possible_labels:
                normalized_label = label
                if label in ['PER']:
                    normalized_label = 'PERSON'
                elif label in ['LOC', 'GPE']:
                    normalized_label = 'LOCATION'
                elif label in ['ORG']:
                    normalized_label = 'ORGANIZATION'
                
                filtered_entities.append({
                    'entity': normalized_label,
                    'word': entity['word'],
                    'start': entity['start'],
                    'end': entity['end'],
                    'score': float(entity['score'])  
                })
        
        print(f"Filtered entities count: {len(filtered_entities)}")
        return filtered_entities
    
    except Exception as e:
        print(f"Error dalam NER: {e}")
        return []


print("Testing fungsi yang diperbaiki:")
test_entities = extract_entities_nusabert_fixed(test_text, pipe)
for entity in test_entities[:10]:  
    print(f"  {entity}")

def merge_consecutive_entities(entities):
    """
    Menggabungkan entity yang berurutan dengan label yang sama
    """
    if not entities:
        return entities
    
    merged = []
    current_entity = entities[0].copy()
    
    for i in range(1, len(entities)):
        next_entity = entities[i]
        
        if (current_entity['entity'] == next_entity['entity'] and 
            current_entity['end'] >= next_entity['start'] - 2): 
            
            current_entity['word'] += ' ' + next_entity['word']
            current_entity['end'] = next_entity['end']
            current_entity['score'] = float((current_entity['score'] + next_entity['score']) / 2) 
        else:
            merged.append(current_entity)
            current_entity = next_entity.copy()
    
    merged.append(current_entity)
    return merged

def extract_all_entities(text, pipe):
    """
    Extract semua entity dari teks (NusaBert + Crime Types)
    """
    nusabert_entities = extract_entities_nusabert_fixed(text, pipe)
    
    crime_entities = detect_crime_types(text)
    
    all_entities = nusabert_entities + crime_entities
    
    all_entities.sort(key=lambda x: x['start'])
    
    nusabert_only = [e for e in all_entities if e['entity'] != 'CRIMETYPE']
    crime_only = [e for e in all_entities if e['entity'] == 'CRIMETYPE']
    
    merged_nusabert = merge_consecutive_entities(nusabert_only)
    
    final_entities = merged_nusabert + crime_only
    final_entities.sort(key=lambda x: x['start'])   
    
    return final_entities

def process_dataframe_entities(df, pipe, sample_size=None):
    """
    Proses seluruh dataframe untuk extract entities
    """
    if sample_size:
        df_sample = df.head(sample_size)
    else:
        df_sample = df
    
    results = []
    
    for idx, row in df_sample.iterrows():
        print(f"Processing artikel {idx+1}/{len(df_sample)}")
        
        entities = extract_all_entities(row['isi_berita'], pipe)
        
        entity_dict = defaultdict(list)
        for entity in entities:
            entity_dict[entity['entity']].append({
                'word': entity['word'],
                'start': int(entity['start']),  
                'end': int(entity['end']),      
                'score': float(entity['score'])
            })
        
        result = {
            'index': int(idx), 
            'url': row['url'],
            'judul': row['judul'],
            'tanggal': row['tanggal'],
            'entities': dict(entity_dict),
            'total_entities': len(entities)
        }
        
        results.append(result)
    
    return results

def save_results_to_files(results, base_filename='entity_results'):
    """
    Simpan hasil ke berbagai format file
    """
    with open(f'{base_filename}.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    flattened_data = []
    for result in results:
        base_row = {
            'index': result['index'],
            'url': result['url'],
            'judul': result['judul'],
            'tanggal': result['tanggal'],
            'total_entities': result['total_entities']
        }
        
        for entity_type in ['DATE', 'TIME', 'LOCATION', 'PERSON', 'ORGANIZATION', 'CRIMETYPE']:
            base_row[f'{entity_type.lower()}_count'] = len(result['entities'].get(entity_type, []))
            base_row[f'{entity_type.lower()}_entities'] = '; '.join([e['word'] for e in result['entities'].get(entity_type, [])])
        
        flattened_data.append(base_row)
    
    df_results = pd.DataFrame(flattened_data)
    df_results.to_csv(f'{base_filename}.csv', index=False, encoding='utf-8')
    
    print(f"Results saved to {base_filename}.json and {base_filename}.csv")

print("\nProcessing semua artikel...")
all_results = process_dataframe_entities(df, pipe)
save_results_to_files(all_results, 'all_entity_results')

Testing fungsi yang diperbaiki:
All labels found: {'B-GPE', 'I-PRD', 'B-ORG', 'B-CRD', 'B-DAT', 'B-LOC', 'I-PER', 'B-PRD', 'I-ORG', 'B-PER', 'I-DAT', 'I-LOC'}
Filtered entities count: 13
  {'entity': 'PERSON', 'word': 'ĠAA', 'start': 33, 'end': 36, 'score': 0.9859017133712769}
  {'entity': 'LOCATION', 'word': 'ĠKub', 'start': 65, 'end': 69, 'score': 0.9789507985115051}
  {'entity': 'LOCATION', 'word': 'uran', 'start': 69, 'end': 73, 'score': 0.9747812747955322}
  {'entity': 'LOCATION', 'word': 'ĠCina', 'start': 73, 'end': 78, 'score': 0.9954337477684021}
  {'entity': 'LOCATION', 'word': 'ĠPalembang', 'start': 78, 'end': 88, 'score': 0.5983035564422607}
  {'entity': 'ORGANIZATION', 'word': 'ĠUniversitas', 'start': 222, 'end': 234, 'score': 0.9998620748519897}
  {'entity': 'ORGANIZATION', 'word': 'ĠTaman', 'start': 234, 'end': 240, 'score': 0.9900770783424377}
  {'entity': 'ORGANIZATION', 'word': 'ĠSiswa', 'start': 240, 'end': 246, 'score': 0.9986501336097717}
  {'entity': 'ORGANIZATION'

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


All labels found: {'B-GPE', 'I-PRD', 'B-CRD', 'B-NOR', 'B-FAC', 'B-DAT', 'B-LOC', 'I-PER', 'B-PRD', 'B-PER', 'I-FAC', 'I-GPE', 'I-DAT', 'I-NOR', 'I-QTY', 'I-LOC'}
Filtered entities count: 30
Processing artikel 6/636
All labels found: {'B-GPE', 'B-CRD', 'B-NOR', 'B-DAT', 'B-PRD', 'B-PER', 'I-EVT', 'I-LOC', 'I-PRD', 'I-MON', 'B-TIM', 'I-ORG', 'I-FAC', 'I-NOR', 'I-DAT', 'I-TIM', 'B-MON', 'B-QTY', 'B-LAW', 'I-QTY', 'B-FAC', 'B-LOC', 'B-ORG', 'B-EVT', 'I-PER', 'I-GPE', 'I-LAW'}
Filtered entities count: 71
Processing artikel 7/636
All labels found: {'B-GPE', 'I-PRD', 'B-ORG', 'B-CRD', 'B-NOR', 'B-FAC', 'B-DAT', 'B-LOC', 'I-PER', 'B-PRD', 'I-ORG', 'B-PER', 'I-FAC', 'I-GPE', 'I-NOR', 'I-DAT', 'I-LOC'}
Filtered entities count: 32
Processing artikel 8/636
All labels found: {'B-GPE', 'I-PRD', 'I-MON', 'B-CRD', 'B-NOR', 'B-EVT', 'B-DAT', 'I-PER', 'B-LAW', 'B-PER', 'I-GPE', 'I-LAW', 'I-DAT', 'I-NOR', 'B-MON', 'B-QTY', 'I-QTY'}
Filtered entities count: 27
Processing artikel 9/636
All labels found: {

In [8]:
output = pd.read_csv('/kaggle/working/all_entity_results.csv')

In [9]:
output.head()

,index,url,judul,tanggal,total_entities,date_count,date_entities,time_count,time_entities,location_count,location_entities,person_count,person_entities,organization_count,organization_entities,crimetype_count,crimetype_entities
0,0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,8,0,NaN,0,NaN,1,ĠKub uran ĠCina ĠPalembang,3,ĠAA; ĠDr ĠAzwar ĠAgus; ĠAzwar,1,ĠUniversitas ĠTaman ĠSiswa ĠPalembang,3,pembunuhan; pemerkosaan; pembunuhan
1,1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,16,0,NaN,0,NaN,1,ĠKota ĠProbolinggo,5,ĠAKBP ĠOki ĠA had ian ĠPurw ono; ĠOki; ĠOki; Ġ...,0,NaN,10,pembunuhan; kasus sabu; pembunuhan; penggelapa...
2,2,https://www.detik.com/jatim/hukum-dan-kriminal...,"Kriminalitas Surabaya Marak, Pengamat Sebut Pe...",2024-09-05 02:01:00,12,0,NaN,0,NaN,2,ĠSurabaya; ĠSurabaya,6,ĠProf .D r ĠS olah udin; ĠS olah udin; ĠS olah...,2,ĠAhli Ġpidana ĠUniversitas ĠBhayangkara Ġ( U b...,2,gangster; begal
3,3,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,12,0,NaN,0,NaN,4,ĠCengkareng; ĠCengkareng; ĠCengkareng ĠJakarta...,6,ĠMAS; ĠA; ĠKombes ĠM ĠSyah d udd i; ĠSyah d ud...,1,Ġdetik News,1,penyiraman air keras
4,4,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,7,0,NaN,0,NaN,1,ĠDusun ĠIII ĠNag ori ĠB ahl iran ĠSib or na ĠK...,6,ĠHermansyah ĠP ohan; ĠZ S; ĠAKP ĠGh ulam ĠYanu...,0,NaN,0,NaN


In [10]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  636 non-null    int64  
 1   url                    636 non-null    object 
 2   judul                  636 non-null    object 
 3   tanggal                636 non-null    object 
 4   total_entities         636 non-null    int64  
 5   date_count             636 non-null    int64  
 6   date_entities          0 non-null      float64
 7   time_count             636 non-null    int64  
 8   time_entities          0 non-null      float64
 9   location_count         636 non-null    int64  
 10  location_entities      628 non-null    object 
 11  person_count           636 non-null    int64  
 12  person_entities        634 non-null    object 
 13  organization_count     636 non-null    int64  
 14  organization_entities  354 non-null    object 
 15  crimet

In [11]:
def auto_evaluate_crimetype_performance(df, sample_size=20):
    """
    Evaluasi otomatis tanpa input manual
    """
    sample_df = df.sample(n=sample_size, random_state=42)
    
    results = []
    
    for idx, row in sample_df.iterrows():
        text = row['isi_berita']
        detected_crimes = detect_crime_types(text)
        
        print(f"\n--- Artikel {idx} ---")
        print(f"Judul: {row['judul']}")
        print(f"Text preview: {text[:200]}...")
        print(f"Detected crimes: {[crime['word'] for crime in detected_crimes]}")
        
        text_lower = text.lower()
        crime_keywords = ['polisi', 'tersangka', 'korban', 'kejahatan', 'kriminal']
        should_have_crime = any(keyword in text_lower for keyword in crime_keywords)
        
        auto_evaluation = "relevant" if should_have_crime else "not_relevant"
        
        results.append({
            'index': idx,
            'detected_crimes': [crime['word'] for crime in detected_crimes],
            'auto_evaluation': auto_evaluation,
            'has_crime_keywords': should_have_crime,
            'text_preview': text[:200]
        })
    
    return results

# Jalankan evaluasi otomatis
auto_eval = auto_evaluate_crimetype_performance(df, 20)


--- Artikel 274 ---
Judul: Marbot Cabuli Bocah Laki-Laki di Simalungun Tersangka, Terancam 15 Tahun Bui
Text preview: Polisi menetapkan Zulham Nasution (24) marbot masjid yang mencabuli bocah laki-laki berusia 12 tahun di Kabupaten Simalungun, Sumatera Utara (Sumut), sebagai tersangka. Pelaku terancam hukuman 15 tahu...
Detected crimes: ['pelecehan seksual']

--- Artikel 631 ---
Judul: Curi Mobil Ambulans Puskesmas, 2 Pria di Simalungun Diringkus Polisi
Text preview: Dua pria di Simalungun ditangkap personel kepolisian karena mencuri mobil ambulans Puskesmas Marubun Jaya. Mobil ambulans tersebut merupakan kendaraan dinas milik Pemkab Simalungun. Penangkapan pencur...
Detected crimes: ['pencurian', 'pencurian', 'pencurian']

--- Artikel 82 ---
Judul: Warga Labuhanbatu Dirampok 2 Pria saat Buang Sampah, HP-Uang Rp 2,2 Juta Raib
Text preview: Seorang pria warga Kelurahan Sei Berombang, Kecamatan Panai Hilir, Kabupaten Labuhanbatu, Sumatera Utara, H (25) dirampok dua pria saat sedang memb

In [ ]:
def analyze_crimetype_statistics(output_df):
    """
    Analisis statistik untuk CRIMETYPE detection
    """
    print("=== CRIMETYPE Detection Statistics ===\n")
    
    total_articles = len(output_df)
    articles_with_crimes = len(output_df[output_df['crimetype_count'] > 0])
    coverage = (articles_with_crimes / total_articles) * 100
    
    print(f"Total Articles: {total_articles}")
    print(f"Articles with Crime Types: {articles_with_crimes}")
    print(f"Coverage: {coverage:.2f}%\n")
    
    crime_counts = output_df['crimetype_count'].value_counts().sort_index()
    print("Crime Count Distribution:")
    for count, freq in crime_counts.items():
        print(f"  {count} crimes: {freq} articles ({freq/total_articles*100:.1f}%)")
    
    all_crimes = []
    for crimes_str in output_df['crimetype_entities'].dropna():
        if crimes_str and crimes_str != '':
            crimes = [crime.strip() for crime in crimes_str.split(';')]
            all_crimes.extend(crimes)
    
    if all_crimes:
        from collections import Counter
        crime_freq = Counter(all_crimes)
        print(f"\nMost Common Crime Types (Top 10):")
        for crime, freq in crime_freq.most_common(10):
            print(f"  {crime}: {freq} times")
    
    no_crimes = output_df[output_df['crimetype_count'] == 0]
    print(f"\nArticles without detected crimes: {len(no_crimes)}")
    
    return {
        'coverage': coverage,
        'crime_distribution': crime_counts.to_dict(),
        'most_common_crimes': crime_freq.most_common(10) if all_crimes else [],
        'articles_without_crimes': len(no_crimes)
    }

# Analisis statistik
stats = analyze_crimetype_statistics(output)

=== CRIMETYPE Detection Statistics ===

Total Articles: 636
Articles with Crime Types: 470
Coverage: 73.90%

Crime Count Distribution:
  0 crimes: 166 articles (26.1%)
  1 crimes: 128 articles (20.1%)
  2 crimes: 116 articles (18.2%)
  3 crimes: 68 articles (10.7%)
  4 crimes: 46 articles (7.2%)
  5 crimes: 35 articles (5.5%)
  6 crimes: 18 articles (2.8%)
  7 crimes: 15 articles (2.4%)
  8 crimes: 6 articles (0.9%)
  9 crimes: 13 articles (2.0%)
  10 crimes: 8 articles (1.3%)
  11 crimes: 4 articles (0.6%)
  12 crimes: 5 articles (0.8%)
  13 crimes: 8 articles (1.3%)

Most Common Crime Types (Top 10):
  pencurian: 347 times
  tawuran: 123 times
  penganiayaan: 123 times
  pembunuhan: 117 times
  begal: 110 times
  curanmor: 85 times
  perampokan: 62 times
  penikaman: 54 times
  pengeroyokan: 51 times
  penembakan: 50 times

Articles without detected crimes: 166


In [13]:
def validate_crimetype_detection(df, output_df):
    """
    Validasi dengan mencari kata kunci crime yang mungkin terlewat
    """
    # Kata kunci tambahan yang mungkin terlewat
    additional_keywords = [
        'kejahatan', 'kriminal', 'tersangka', 'terdakwa', 'pelaku',
        'korban', 'kepolisian', 'penangkapan', 'penahanan',
        'sidang', 'vonis', 'hukuman', 'denda'
    ]
    
    false_negatives = []
    
    for idx, row in df.iterrows():
        text = row['isi_berita'].lower()
        output_row = output_df[output_df['index'] == idx]
        
        if len(output_row) == 0:
            continue
            
        detected_crimes = output_row.iloc[0]['crimetype_count']
        
        has_crime_keywords = any(keyword in text for keyword in additional_keywords)
        
        if has_crime_keywords and detected_crimes == 0:
            false_negatives.append({
                'index': idx,
                'judul': row['judul'],
                'keywords_found': [kw for kw in additional_keywords if kw in text]
            })
    
    print(f"Potential False Negatives: {len(false_negatives)}")
    for fn in false_negatives[:10]:  
        print(f"  Article {fn['index']}: {fn['judul']}")
        print(f"    Keywords: {fn['keywords_found']}")
    
    return false_negatives

# Validasi false negatives
false_negs = validate_crimetype_detection(df, output)

Potential False Negatives: 159
  Article 4: Hilang Nyawa Pria di Simalungun gegara Rebutan Mikrofon di Warung Tuak
    Keywords: ['tersangka', 'pelaku', 'korban', 'kepolisian']
  Article 8: Anehnya Gani, Ngaku Dikejar Hantu hingga Tega Bacok Tetangga
    Keywords: ['kriminal', 'pelaku', 'kepolisian', 'sidang']
  Article 10: Ngeri! Mahasiswa PKL di Bawaslu Lampung Ditembak OTK
    Keywords: ['korban', 'kepolisian']
  Article 13: Bocah SD di Medan Diperkosa Pria di Dekat Kuburan, Modus Kenal Ortu Korban
    Keywords: ['pelaku', 'korban', 'kepolisian']
  Article 14: Tak Diberi Jalan Saat Menyalip, Sopir Elf Bacok Teman di Garut
    Keywords: ['pelaku', 'korban']
  Article 18: Serang Polisi Saat Ditangkap, Kakek Bawa 10 Kg Sabu Ditembak Bareng 2 Teman
    Keywords: ['tersangka', 'pelaku', 'kepolisian', 'hukuman']
  Article 23: 4 Pedagang Miras Ilegal di Jombang Diringkus, 683 Botol Disita
    Keywords: ['kriminal']
  Article 24: Pria Kendari Tewas Bersimbah Darah Dibacok Berkali-kali, Pela

In [ ]:
def estimate_precision_recall(df, output_df, sample_size=100):
    """
    Estimasi precision dan recall berdasarkan sample
    """
    sample_indices = np.random.choice(len(df), sample_size, replace=False)
    
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for idx in sample_indices:
        text = df.iloc[idx]['isi_berita']
        output_row = output_df[output_df['index'] == idx]
        
        if len(output_row) == 0:
            continue
            
        detected_crimes = output_row.iloc[0]['crimetype_entities']
        detected_count = output_row.iloc[0]['crimetype_count']
        
        text_lower = text.lower()
        
        # Heuristic: jika ada kata-kata ini, seharusnya ada crime
        crime_indicators = ['polisi', 'tersangka', 'korban', 'kejahatan', 'kriminal']
        should_have_crime = any(indicator in text_lower for indicator in crime_indicators)
        
        if detected_count > 0 and should_have_crime:
            true_positives += 1
        elif detected_count > 0 and not should_have_crime:
            false_positives += 1
        elif detected_count == 0 and should_have_crime:
            false_negatives += 1
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"Estimated Performance:")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall: {recall:.3f}")
    print(f"  F1-Score: {f1:.3f}")
    
    return {'precision': precision, 'recall': recall, 'f1': f1}

# Estimasi performance
performance = estimate_precision_recall(df, output)

Estimated Performance:
  Precision: 0.987
  Recall: 0.786
  F1-Score: 0.875


In [15]:
def analyze_error_cases(df, output_df):
    """
    Analisis kasus-kasus error dalam deteksi CRIMETYPE
    """
    print("=== Error Case Analysis ===\n")
    
    # 1. Articles dengan crime count tinggi (potential false positives)
    high_crime_articles = output_df[output_df['crimetype_count'] > 3].sort_values('crimetype_count', ascending=False)
    
    print("Articles with >3 crime types (potential over-detection):")
    for _, row in high_crime_articles.head(5).iterrows():
        print(f"  Article {row['index']}: {row['crimetype_count']} crimes")
        print(f"    Crimes: {row['crimetype_entities']}")
        print(f"    Title: {row['judul']}")
        print()
    
    # 2. Check for duplicate crime detection
    duplicate_crimes = []
    for _, row in output_df.iterrows():
        if pd.notna(row['crimetype_entities']) and row['crimetype_entities']:
            crimes = [c.strip() for c in row['crimetype_entities'].split(';')]
            if len(crimes) != len(set(crimes)):  # Has duplicates
                duplicate_crimes.append(row['index'])
    
    print(f"Articles with duplicate crime detection: {len(duplicate_crimes)}")
    
    return {
        'high_crime_articles': high_crime_articles,
        'duplicate_crime_articles': duplicate_crimes
    }

# Analisis error cases
errors = analyze_error_cases(df, output)

=== Error Case Analysis ===

Articles with >3 crime types (potential over-detection):
  Article 34: 13 crimes
    Crimes: pencurian; curanmor; penganiayaan; pencurian; curanmor; pencurian; curanmor; curanmor; pencurian; curanmor; curanmor; curanmor; pencurian
    Title: 70 Tersangka Pencurian-Pengeroyokan di Sidoarjo Diringkus Selama 12 Hari

  Article 619: 13 crimes
    Crimes: perampokan; pencurian; perampokan; pencurian; perampokan; pencurian; pencurian dengan kekerasan; pencurian; pencurian; pencurian; perampokan; pencurian; pencurian dengan kekerasan
    Title: 5 Fakta Ngeri Perampokan Agen Perbankan di Bandung

  Article 542: 13 crimes
    Crimes: pencurian; curanmor; kasus narkoba; curanmor; pencurian; curanmor; pencurian; curanmor; pencurian; kasus narkoba; pencurian; pencurian motor; kasus narkoba
    Title: Kasus Narkoba-Curanmor, Polres Tabanan Ringkus 16 Orang

  Article 310: 13 crimes
    Crimes: pencurian; pencurian; penganiayaan; pencurian; pencurian; pencurian; pencuria

In [16]:
# Quick check untuk melihat performa
print("=== Quick CRIMETYPE Performance Check ===")
analyze_crimetype_statistics(output)
print("\n" + "="*50)
validate_crimetype_detection(df, output)

=== Quick CRIMETYPE Performance Check ===
=== CRIMETYPE Detection Statistics ===

Total Articles: 636
Articles with Crime Types: 470
Coverage: 73.90%

Crime Count Distribution:
  0 crimes: 166 articles (26.1%)
  1 crimes: 128 articles (20.1%)
  2 crimes: 116 articles (18.2%)
  3 crimes: 68 articles (10.7%)
  4 crimes: 46 articles (7.2%)
  5 crimes: 35 articles (5.5%)
  6 crimes: 18 articles (2.8%)
  7 crimes: 15 articles (2.4%)
  8 crimes: 6 articles (0.9%)
  9 crimes: 13 articles (2.0%)
  10 crimes: 8 articles (1.3%)
  11 crimes: 4 articles (0.6%)
  12 crimes: 5 articles (0.8%)
  13 crimes: 8 articles (1.3%)

Most Common Crime Types (Top 10):
  pencurian: 347 times
  tawuran: 123 times
  penganiayaan: 123 times
  pembunuhan: 117 times
  begal: 110 times
  curanmor: 85 times
  perampokan: 62 times
  penikaman: 54 times
  pengeroyokan: 51 times
  penembakan: 50 times

Articles without detected crimes: 166

Potential False Negatives: 159
  Article 4: Hilang Nyawa Pria di Simalungun gegar

[{'index': 4,
  'judul': 'Hilang Nyawa Pria di Simalungun gegara Rebutan Mikrofon di Warung Tuak',
  'keywords_found': ['tersangka', 'pelaku', 'korban', 'kepolisian']},
 {'index': 8,
  'judul': 'Anehnya Gani, Ngaku Dikejar Hantu hingga Tega Bacok Tetangga',
  'keywords_found': ['kriminal', 'pelaku', 'kepolisian', 'sidang']},
 {'index': 10,
  'judul': 'Ngeri! Mahasiswa PKL di Bawaslu Lampung Ditembak OTK',
  'keywords_found': ['korban', 'kepolisian']},
 {'index': 13,
  'judul': 'Bocah SD di Medan Diperkosa Pria di Dekat Kuburan, Modus Kenal Ortu Korban',
  'keywords_found': ['pelaku', 'korban', 'kepolisian']},
 {'index': 14,
  'judul': 'Tak Diberi Jalan Saat Menyalip, Sopir Elf Bacok Teman di Garut',
  'keywords_found': ['pelaku', 'korban']},
 {'index': 18,
  'judul': 'Serang Polisi Saat Ditangkap, Kakek Bawa 10 Kg Sabu Ditembak Bareng 2 Teman',
  'keywords_found': ['tersangka', 'pelaku', 'kepolisian', 'hukuman']},
 {'index': 23,
  'judul': '4 Pedagang Miras Ilegal di Jombang Diringkus,

In [17]:
def hybrid_crime_detection(text, pipe):
    """
    Kombinasi NusaBert + Enhanced Pattern Matching
    """
    nusabert_entities = pipe(text)
    crime_from_nusabert = [e for e in nusabert_entities if 'crime' in e['entity'].lower()]
    
    pattern_crimes = enhanced_crime_detection(text)
    
    all_crimes = crime_from_nusabert + pattern_crimes
    
    return all_crimes

In [18]:
def calculate_actual_f1_score(df, pipe, sample_size=50):
    """
    Menghitung F1 score yang akurat dengan manual validation
    """
    # Ambil sample untuk manual validation
    sample_df = df.sample(n=sample_size, random_state=42)
    
    y_true = []
    y_pred = []
    
    print("Manual validation diperlukan untuk setiap artikel:")
    print("Ketik 1 jika artikel berisi crime, 0 jika tidak")
    
    for idx, row in sample_df.iterrows():
        text = row['isi_berita']
        
        # Deteksi menggunakan model current
        detected_crimes = detect_crime_types(text)
        has_detected_crime = len(detected_crimes) > 0
        
        print(f"\n--- Artikel {idx} ---")
        print(f"Judul: {row['judul']}")
        print(f"Preview: {text[:200]}...")
        print(f"Detected crimes: {[c['word'] for c in detected_crimes]}")
        
        # Manual input (untuk demo, kita bisa automate dengan heuristic)
        # manual_label = int(input("Apakah artikel ini benar mengandung crime? (1/0): "))
        
        # Temporary heuristic untuk demo
        crime_keywords = ['polisi', 'tersangka', 'korban', 'ditangkap', 'kejahatan']
        manual_label = 1 if any(kw in text.lower() for kw in crime_keywords) else 0
        
        y_true.append(manual_label)
        y_pred.append(1 if has_detected_crime else 0)
    
    # Hitung metrics
    from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
    
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"\n=== ACTUAL PERFORMANCE ===")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-Score: {f1:.3f}")
    
    print(f"\nDetailed Report:")
    print(classification_report(y_true, y_pred, target_names=['No Crime', 'Crime']))
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'y_true': y_true,
        'y_pred': y_pred
    }

# Implementasi enhanced_crime_detection yang hilang
def enhanced_crime_detection(text):
    """
    Enhanced pattern matching untuk crime detection
    """
    # Expanded crime patterns
    enhanced_patterns = {
        'pembunuhan': [
            r'\b(membunuh|dibunuh|terbunuh|pembunuhan|bunuh)\b',
            r'\b(tewas|meninggal dunia)\b.*\b(dibunuh|terbunuh)\b'
        ],
        'pencurian': [
            r'\b(mencuri|dicuri|tercuri|pencurian|curi|curanmor|begal)\b',
            r'\b(mengambil|diambil)\b.*\b(tanpa izin|secara paksa)\b'
        ],
        'narkoba': [
            r'\b(narkoba|sabu|ganja|ekstasi|kokain|heroin)\b',
            r'\b(obat.{0,10}terlarang|zat.{0,10}adiktif)\b'
        ],
        'korupsi': [
            r'\b(korupsi|suap|gratifikasi)\b',
            r'\b(menyalahgunakan.{0,20}jabatan)\b'
        ]
    }
    
    detected_crimes = []
    text_lower = text.lower()
    
    for crime_type, patterns in enhanced_patterns.items():
        for pattern in patterns:
            matches = re.finditer(pattern, text_lower)
            for match in matches:
                detected_crimes.append({
                    'entity': 'CRIMETYPE',
                    'word': crime_type,
                    'start': match.start(),
                    'end': match.end(),
                    'score': 0.9
                })
    
    return detected_crimes

# Update hybrid function
def hybrid_crime_detection(text, pipe):
    """
    Kombinasi NusaBert + Enhanced Pattern Matching
    """
    nusabert_entities = pipe(text)
    crime_from_nusabert = [e for e in nusabert_entities if 'crime' in e['entity'].lower()]
    
    pattern_crimes = enhanced_crime_detection(text)
    
    # Remove duplicates
    all_crimes = crime_from_nusabert + pattern_crimes
    
    # Simple deduplication based on overlap
    if len(all_crimes) > 1:
        unique_crimes = []
        for crime in all_crimes:
            is_duplicate = False
            for existing in unique_crimes:
                if (abs(crime['start'] - existing['start']) < 10 and 
                    crime['word'].lower() in existing['word'].lower()):
                    is_duplicate = True
                    break
            if not is_duplicate:
                unique_crimes.append(crime)
        return unique_crimes
    
    return all_crimes

# Test hybrid approach
print("Testing hybrid approach...")
test_results = calculate_actual_f1_score(df, pipe, 600)

Testing hybrid approach...
Manual validation diperlukan untuk setiap artikel:
Ketik 1 jika artikel berisi crime, 0 jika tidak

--- Artikel 274 ---
Judul: Marbot Cabuli Bocah Laki-Laki di Simalungun Tersangka, Terancam 15 Tahun Bui
Preview: Polisi menetapkan Zulham Nasution (24) marbot masjid yang mencabuli bocah laki-laki berusia 12 tahun di Kabupaten Simalungun, Sumatera Utara (Sumut), sebagai tersangka. Pelaku terancam hukuman 15 tahu...
Detected crimes: ['pelecehan seksual']

--- Artikel 631 ---
Judul: Curi Mobil Ambulans Puskesmas, 2 Pria di Simalungun Diringkus Polisi
Preview: Dua pria di Simalungun ditangkap personel kepolisian karena mencuri mobil ambulans Puskesmas Marubun Jaya. Mobil ambulans tersebut merupakan kendaraan dinas milik Pemkab Simalungun. Penangkapan pencur...
Detected crimes: ['pencurian', 'pencurian', 'pencurian']

--- Artikel 82 ---
Judul: Warga Labuhanbatu Dirampok 2 Pria saat Buang Sampah, HP-Uang Rp 2,2 Juta Raib
Preview: Seorang pria warga Kelurahan Sei Ber

In [19]:
actual_performance = calculate_actual_f1_score(df, pipe, 30)

def compare_approaches(df, pipe, sample_size=20):
    """
    Bandingkan performa different approaches
    """
    sample_df = df.sample(n=sample_size, random_state=42)
    
    approaches = {
        'pattern_only': [],
        'hybrid': []
    }
    
    for idx, row in sample_df.iterrows():
        text = row['isi_berita']
        
        # Pattern only
        pattern_crimes = detect_crime_types(text)
        approaches['pattern_only'].append(len(pattern_crimes) > 0)
        
        # Hybrid
        hybrid_crimes = hybrid_crime_detection(text, pipe)
        approaches['hybrid'].append(len(hybrid_crimes) > 0)
    
    return approaches

# Test comparison
comparison = compare_approaches(df, pipe, 600)

Manual validation diperlukan untuk setiap artikel:
Ketik 1 jika artikel berisi crime, 0 jika tidak

--- Artikel 274 ---
Judul: Marbot Cabuli Bocah Laki-Laki di Simalungun Tersangka, Terancam 15 Tahun Bui
Preview: Polisi menetapkan Zulham Nasution (24) marbot masjid yang mencabuli bocah laki-laki berusia 12 tahun di Kabupaten Simalungun, Sumatera Utara (Sumut), sebagai tersangka. Pelaku terancam hukuman 15 tahu...
Detected crimes: ['pelecehan seksual']

--- Artikel 631 ---
Judul: Curi Mobil Ambulans Puskesmas, 2 Pria di Simalungun Diringkus Polisi
Preview: Dua pria di Simalungun ditangkap personel kepolisian karena mencuri mobil ambulans Puskesmas Marubun Jaya. Mobil ambulans tersebut merupakan kendaraan dinas milik Pemkab Simalungun. Penangkapan pencur...
Detected crimes: ['pencurian', 'pencurian', 'pencurian']

--- Artikel 82 ---
Judul: Warga Labuhanbatu Dirampok 2 Pria saat Buang Sampah, HP-Uang Rp 2,2 Juta Raib
Preview: Seorang pria warga Kelurahan Sei Berombang, Kecamatan Panai Hil